In [9]:
import inference.InferCode_Inference as II
import data_process.self_supervised.data_reader as data_reader
import xml.etree.ElementTree as ET
import numpy as np
import time
import tqdm
import copy
import math

token_dict_path = "/home/stanley/Desktop/dictionaries/token2id.json"
type_dict_path = "/home/stanley/Desktop/dictionaries/type2id.json"
model_weight_path = "/home/stanley/Desktop/model_weights/epoch_77.pkl"
subtree_count = 635374
dimension = 64

inferencer = II.InferCode_Inference(model_weight_path, token_dict_path, type_dict_path, subtree_count, dimension)

# code examples to get code vector
file_path = "/home/stanley/Desktop/test_20k"
# file_path = "/home/stanley/Desktop/test_dataset_100k"

In [10]:

# construct a tree that has 10% smaller number of nodes
def reduce(tree, percentage):
    reduced_tree = copy.deepcopy(tree)
    node_count = len(reduced_tree.getroot().findall(".//*"))
    threshold = math.ceil(node_count * percentage)
    delete_e = []
    difference = node_count

    for e in reduced_tree.iter():
        num_child = len(e.findall(".//*"))
        if abs(num_child - threshold) < difference :
            delete_e = e
            difference = abs(num_child - threshold)
    delete_e.clear()
    delete_e.tag = "unknown_type"
    delete_e.text = "unknown_token"

    #return 1.0 - len(reduced_tree.getroot().findall(".//*"))/node_count
    return reduced_tree


In [11]:
dataset = data_reader.Data_Reader(file_path).processed_dataset

Data Processing : 100%|██████████| 20705/20705 [00:50<00:00, 409.01it/s]


In [14]:
def collect_file_paths(path):
        '''
        collect individual xml file path and return those as a list
        '''
        file_paths = []
        for root, directories, filenames in os.walk(path):
            for filename in filenames:
                file_paths.append(os.path.join(root,filename))
        return file_paths



In [19]:
model_direct = "/home/stanley/Desktop/model_weights"
model_paths = collect_file_paths(model_direct)
paths = collect_file_paths(file_path)

In [29]:
full_tree = []
tree_10 = []
tree_30 = []

len_full = []
len_10 = []
len_30 = []

for idx, path in tqdm.tqdm(enumerate(paths)):
    original_tree = ET.parse(path)
    full_tree.append(copy.deepcopy(original_tree))
    tree_10.append(reduce(original_tree, 0.1))
    tree_30.append(reduce(original_tree, 0.3))
    len_full.append(len(original_tree.getroot().findall(".//*")))
    len_10.append(len(tree_10[idx].getroot().findall(".//*")))
    len_30.append(len(tree_30[idx].getroot().findall(".//*")))

20705it [02:22, 144.85it/s]


In [31]:
for model_path in model_paths:
    inferencer = II.InferCode_Inference(model_path, token_dict_path, type_dict_path, subtree_count, dimension)

    positive_count = 0
    total = len(paths)
    difference = 0.1 # 10%

    for idx, path in tqdm.tqdm(enumerate(paths)):
        
        if len_10[idx]/len_full[idx] - len_30[idx]/len_full[idx] < difference: # remove examples that have a small difference in removed nummber of nodes

            total -= 1

        else:

            code_vector = inferencer.code2vec(copy.deepcopy(full_tree[idx]), tree = True)

            code_vector_10 = inferencer.code2vec(copy.deepcopy(tree_10[idx]), tree = True)

            code_vector_30 = inferencer.code2vec(copy.deepcopy(tree_30[idx]), tree = True)
            
            if np.linalg.norm(code_vector-code_vector_10) < np.linalg.norm(code_vector-code_vector_30):
                positive_count += 1
        
    print(model_path, " has accuracy :", positive_count / total, " The total number is ", total)

1it [00:00, 253.48it/s]


RuntimeError: Could not infer dtype of NoneType